In [505]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris, load_breast_cancer, load_wine
from sklearn.model_selection import train_test_split
from xgboost import plot_tree
from z3 import *
import numpy as np
import pandas as pd

In [506]:
set_option(rational_to_decimal=True)

# model

In [507]:
# import sys
# import os
# sys.path.append(os.path.abspath('../../../'))

# from model.xai_xgb_z3 import XGBoostExplainer

In [508]:
from z3 import *
import numpy as np

class XGBoostExplainer:
    """Apenas classificação binária e base_score = None
    data = X. labels = y
    """

    def __init__(self, model, data, labels):
        """_summary_

        Args:
            model (XGBoost): xgboost model fited
            data (DataFrame): dataframe (X or X_train)
            labels (array): y (targets)
        """
        self.model = model
        self.data = data.values
        self.columns = data.columns
        self.T_constraints = self.feature_constraints_expression(self.data)
        self.T_model = self.model_trees_expression(self.model)
        self.T = And(self.T_model, self.T_constraints)
        self.label_proportions = labels.mean()

    def explain(self, instance, reorder="asc"):
        self.D = self.decision_function_expression(
            self.model, [instance], self.label_proportions
        )
        self.I = self.instance_expression(instance)

        return self.explain_expression(self.I, self.T, self.D, self.model, reorder)

    def feature_constraints_expression(self, X: np.ndarray):
        """
        Recebe um dataset e retorna uma fórmula no z3 com:
        - Restrições de valor máximo e mínimo para features contínuas.
        - Restrições de igualdade para features categóricas binárias.
        """
        constraints = []

        for i in range(X.shape[1]):
            feature_values = X[:, i]
            unique_values = np.unique(feature_values)

            x = Real(self.columns[i])

            if len(unique_values) == 2:
                a, b = unique_values
                constraint = Or(x == RealVal(a), x == RealVal(b))
            else:
                min_val, max_val = feature_values.min(), feature_values.max()
                min_val_z3 = RealVal(min_val)
                max_val_z3 = RealVal(max_val)
                constraint = And(min_val_z3 <= x, x <= max_val_z3)

            constraints.append(constraint)

        return And(*constraints)

    def model_trees_expression(self, model):
        """
        Constrói expressões lógicas para todas as árvores de decisão em um dataframe de XGBoost.
        Para árvores que são apenas folhas, gera diretamente um And com o valor da folha.

        Args:
            df (pd.DataFrame): Dataframe contendo informações das árvores.
            class_index (int): Índice da classe atual.

        Returns:
            z3.ExprRef: Fórmula representando todos os caminhos de todas as árvores.
        """
        df = model.get_booster().trees_to_dataframe()
        class_index = 0  # if model.n_classes_ == 2:

        all_tree_formulas = []

        for tree_index in df["Tree"].unique():
            tree_df = df[df["Tree"] == tree_index]
            o = Real(f"o_{tree_index}_{class_index}")

            if len(tree_df) == 1 and tree_df.iloc[0]["Feature"] == "Leaf":
                leaf_value = tree_df.iloc[0]["Gain"]
                all_tree_formulas.append(And(o == leaf_value))
                continue

            path_formulas = []

            def get_conditions(node_id):
                conditions = []
                current_node = tree_df[tree_df["ID"] == node_id]
                if current_node.empty:
                    return conditions

                parent_node = tree_df[
                    (tree_df["Yes"] == node_id) | (tree_df["No"] == node_id)
                ]
                if not parent_node.empty:
                    parent_data = parent_node.iloc[0]
                    feature = parent_data["Feature"]
                    split_value = parent_data["Split"]
                    x = Real(feature)
                    if parent_data["Yes"] == node_id:
                        conditions.append(x < split_value)
                    else:
                        conditions.append(x >= split_value)
                    conditions = get_conditions(parent_data["ID"]) + conditions

                return conditions

            for _, node in tree_df[tree_df["Feature"] == "Leaf"].iterrows():
                leaf_value = node["Gain"]
                leaf_id = node["ID"]
                conditions = get_conditions(leaf_id)
                path_formula = And(*conditions)
                implication = Implies(path_formula, o == leaf_value)
                path_formulas.append(implication)

            all_tree_formulas.append(And(*path_formulas))

        return And(*all_tree_formulas)

    def decision_function_expression(self, model, x, label_proportions):
        n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
        predicted_class = model.predict(x)[0]
        init_value = label_proportions

        equation_list = []
        for class_number in range(n_classes):
            estimator_list = []
            for estimator_number in range(
                int(len(model.get_booster().get_dump()) / n_classes)
            ):
                o = Real(f"o_{estimator_number}_{class_number}")
                estimator_list.append(o)
            equation_o = Sum(estimator_list) + init_value
            equation_list.append(equation_o)

        if n_classes <= 2:
            if predicted_class == 0:
                final_equation = equation_list[0] < 0
            else:
                final_equation = equation_list[0] > 0
        else:
            compare_equation = []
            for class_number in range(n_classes):
                if predicted_class != class_number:
                    compare_equation.append(
                        equation_list[predicted_class] > equation_list[class_number]
                    )
            final_equation = compare_equation

        return And(final_equation)

    def instance_expression(self, instance):
        formula = [Real(self.columns[i]) == value for i, value in enumerate(instance)]
        return formula

    def explain_expression(self, I, T, D, model, reorder):
        X = I.copy()
        T_s = simplify(T)
        D_s = simplify(D)

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            X = [X[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            X = [X[i] for i in sorted_feature_indices]

        for feature in X.copy():
            X.remove(feature)

            # prove(Implies(And(And(X), T), D))
            if self.is_proved(Implies(And(And(X), T_s), D_s)):
                continue
                # print('proved')
            else:
                # print('not proved')
                X.append(feature)

        return X

    def is_proved(self, f):
        s = Solver()
        s.add(Not(f))
        if s.check() == unsat:
            return True
        else:
            return False

    def delta_expression(self, exp):
        expressions = []
        delta = Real('delta')

        for name in exp:
            tokens = name.split(" == ")
            z3feature = Real(tokens[0])
            value = tokens[1]
            expressions.append(z3feature >= float(value) - delta)
            expressions.append(z3feature <= float(value) + delta)  

        expressions.append(delta >= 0)
        return expressions
    
    def explain_range(self, instance, reorder="asc"):
        set_option(rational_to_decimal=True) # fix this
        
        exp = self.explain(instance, reorder)
        if exp != []:
            expstr = []
            for expression in exp:
                expstr.append(str(expression))
            self.delta_expressions = self.delta_expression(expstr)
            
            opt = Optimize()
            opt.add(self.delta_expressions)
            opt.add(self.T)
            opt.add(Not(self.D))

            delta = Real('delta')
            expmin = opt.minimize(delta)
            opt.check()

            value = str(expmin.value())

            if "+ epsilon" in value:
                numeric_value = float(value.split(" + ")[0])
            else:
                numeric_value = float(value) - 0.01
            
            if numeric_value > 0.01:
                return numeric_value
        
        return 0

# pred

In [509]:
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

y = np.where(y == 0, 0, 1) # converte em binario
# X = X.iloc[:, :2] # corta colunas do df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,  random_state=101)

xgbc = XGBClassifier(n_estimators=20, max_depth=5, learning_rate=0.1)
xgbc.fit(X_train, y_train)

preds = xgbc.predict(X_test)
preds

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0])

In [510]:
booster = xgbc.get_booster()
booster

In [511]:
# for i in range(3):
#        plot_tree(xgbc, num_trees=i)

In [512]:
trees = booster.get_dump(with_stats=True)
for i, tree in enumerate(trees):
  print(f"Tree {i}:")
  print(tree)

Tree 0:
0:[proline<770] yes=1,no=2,missing=2,gain=98.9444351,cover=31.1348114
	1:leaf=0.140779421,cover=19.2948132
	2:[flavanoids<2.28999996] yes=3,no=4,missing=4,gain=21.9696541,cover=11.8399982
		3:leaf=0.0646421313,cover=1.97333312
		4:leaf=-0.270458966,cover=9.86666584

Tree 1:
0:[proline<770] yes=1,no=2,missing=2,gain=80.0747452,cover=30.8850746
	1:leaf=0.134412304,cover=18.2871494
	2:[flavanoids<2.17000008] yes=3,no=4,missing=4,gain=16.8105927,cover=12.5979252
		3:leaf=0.0870177448,cover=1.49905205
		4:[magnesium<126] yes=5,no=6,missing=6,gain=1.48054123,cover=11.0988731
			5:leaf=-0.236294851,cover=9.9362278
			6:leaf=-0.0528627001,cover=1.16264558

Tree 2:
0:[proline<770] yes=1,no=2,missing=2,gain=66.4681015,cover=30.2188606
	1:leaf=0.129026279,cover=17.2411041
	2:[flavanoids<2.17000008] yes=3,no=4,missing=4,gain=13.5625763,cover=12.9777565
		3:leaf=0.0835888311,cover=1.44813657
		4:[color_intensity<3.74000001] yes=5,no=6,missing=6,gain=1.46408463,cover=11.5296202
			5:leaf=-0.

In [513]:
booster.get_dump()

['0:[proline<770] yes=1,no=2,missing=2\n\t1:leaf=0.140779421\n\t2:[flavanoids<2.28999996] yes=3,no=4,missing=4\n\t\t3:leaf=0.0646421313\n\t\t4:leaf=-0.270458966\n',
 '0:[proline<770] yes=1,no=2,missing=2\n\t1:leaf=0.134412304\n\t2:[flavanoids<2.17000008] yes=3,no=4,missing=4\n\t\t3:leaf=0.0870177448\n\t\t4:[magnesium<126] yes=5,no=6,missing=6\n\t\t\t5:leaf=-0.236294851\n\t\t\t6:leaf=-0.0528627001\n',
 '0:[proline<770] yes=1,no=2,missing=2\n\t1:leaf=0.129026279\n\t2:[flavanoids<2.17000008] yes=3,no=4,missing=4\n\t\t3:leaf=0.0835888311\n\t\t4:[color_intensity<3.74000001] yes=5,no=6,missing=6\n\t\t\t5:leaf=-0.0430838913\n\t\t\t6:leaf=-0.20804821\n',
 '0:[proline<770] yes=1,no=2,missing=2\n\t1:leaf=0.124404095\n\t2:[flavanoids<2.17000008] yes=3,no=4,missing=4\n\t\t3:leaf=0.080420211\n\t\t4:[color_intensity<3.74000001] yes=5,no=6,missing=6\n\t\t\t5:leaf=-0.0405692905\n\t\t\t6:leaf=-0.186384767\n',
 '0:[proline<770] yes=1,no=2,missing=2\n\t1:leaf=0.120385185\n\t2:[flavanoids<2.28999996] yes=

In [514]:
print(type(booster))

<class 'xgboost.core.Booster'>


In [515]:
X_test.values[0]

array([1.305e+01, 1.650e+00, 2.550e+00, 1.800e+01, 9.800e+01, 2.450e+00,
       2.430e+00, 2.900e-01, 1.440e+00, 4.250e+00, 1.120e+00, 2.510e+00,
       1.105e+03])

# explain

In [516]:
explainer = XGBoostExplainer(xgbc, X, y)

In [517]:
i = 2
print(explainer.explain(X_test.values[i]))
print('delta=', explainer.explain_range(X_test.values[i]))

[proline == 650]
delta= 119.99


In [518]:
deltatest = 120
valormaior = xgbc.predict(np.array([1.305e+01, 1.650e+00, 2.550e+00, 1.800e+01, 9.800e+01, 2.450e+00,
        2.430e+00, 2.900e-01, 1.440e+00, 4.250e+00, 1.120e+00, 2.510e+00,
        650 + deltatest]).reshape(1, -1))
valormenor = xgbc.predict(np.array([1.305e+01, 1.650e+00, 2.550e+00, 1.800e+01, 9.800e+01, 2.450e+00,
        2.430e+00, 2.900e-01, 1.440e+00, 4.250e+00, 1.120e+00, 2.510e+00,
        650 - deltatest]).reshape(1, -1))

print(valormaior, valormenor)

[0] [1]


In [519]:
for i in range(len(X_test)):
    print('features', len(explainer.explain(X_test.values[i])), '| delta=', explainer.explain_range(X_test.values[i]))

features 2 | delta= 0.14000004
features 3 | delta= 0.40000004
features 1 | delta= 119.99
features 2 | delta= 1.27000004
features 1 | delta= 94.99
features 1 | delta= 269.99
features 1 | delta= 19.99
features 2 | delta= 0.47000004
features 1 | delta= 89.99
features 1 | delta= 109.99
features 2 | delta= 0.90000004
features 1 | delta= 91.99
features 1 | delta= 89.99
features 2 | delta= 0.39000004
features 1 | delta= 9.99
features 2 | delta= 0.50000004
features 1 | delta= 205.99
features 1 | delta= 249.99
features 1 | delta= 139.99
features 1 | delta= 144.99
features 1 | delta= 404.99
features 1 | delta= 219.99
features 1 | delta= 179.99
features 1 | delta= 1.47000008
features 2 | delta= 0.85000004
features 2 | delta= 0.63000004
features 1 | delta= 391.99
features 1 | delta= 162.99
features 1 | delta= 354.99
features 1 | delta= 319.99
features 1 | delta= 289.99
features 1 | delta= 397.99
features 1 | delta= 319.99
features 2 | delta= 0.24000004
features 1 | delta= 34.99
features 2 | delta=

In [520]:
# count = 0
# for i in range(len(X_test)):
#     delt1 = explainer.explain_range(X_test.values[i])
#     valor = float(X_test['petal length (cm)'].values[i])

#     valormaior = xgbc.predict(np.array([0 , 0, valor + delt1, 0]).reshape(1, -1))
#     valormenor = xgbc.predict(np.array([0 , 0, valor - delt1, 0]).reshape(1, -1))
    
#     if valormenor != valormaior:
#         count += 1
# print(count)

In [521]:
X.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'],
      dtype='object')

In [522]:
print(X_test.values[0])
print(y_test[0])
print(xgbc.predict([X_test.values[0]]))

[1.305e+01 1.650e+00 2.550e+00 1.800e+01 9.800e+01 2.450e+00 2.430e+00
 2.900e-01 1.440e+00 4.250e+00 1.120e+00 2.510e+00 1.105e+03]
0
[0]


In [523]:
# filter_condition = X["petal length (cm)"] == 1.4
# filtered_X = X[filter_condition]
# filtered_y = y[filter_condition]
# print("Entradas em X:")
# print(filtered_X)

# print("\nCorrespondências em y:")
# print(filtered_y)

# print("\n Predictions")
# print(xgbc.predict(filtered_X))

## print expresions

In [524]:
xgbc.get_booster().trees_to_dataframe()

,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover,Category
0,0,0,0-0,proline,770.00,0-1,0-2,0-2,98.944435,31.134811,NaN
1,0,1,0-1,Leaf,NaN,NaN,NaN,NaN,0.140779,19.294813,NaN
2,0,2,0-2,flavanoids,2.29,0-3,0-4,0-4,21.969654,11.839998,NaN
3,0,3,0-3,Leaf,NaN,NaN,NaN,NaN,0.064642,1.973333,NaN
4,0,4,0-4,Leaf,NaN,NaN,NaN,NaN,-0.270459,9.866666,NaN
...,...,...,...,...,...,...,...,...,...,...,...
127,19,2,19-2,Leaf,NaN,NaN,NaN,NaN,-0.094247,3.920163,NaN
128,19,3,19-3,Leaf,NaN,NaN,NaN,NaN,0.088128,5.392917,NaN
129,19,4,19-4,alcalinity_of_ash,19.50,19-5,19-6,19-6,0.587271,2.180460,NaN
130,19,5,19-5,Leaf,NaN,NaN,NaN,NaN,-0.053039,1.015403,NaN


In [525]:
print(explainer.T_constraints)

And(And(11.03 <= alcohol, 14.83 >= alcohol),
    And(0.74 <= malic_acid, 5.8 >= malic_acid),
    And(1.36 <= ash, 3.23 >= ash),
    And(10.6 <= alcalinity_of_ash, 30 >= alcalinity_of_ash),
    And(70 <= magnesium, 162 >= magnesium),
    And(0.98 <= total_phenols, 3.88 >= total_phenols),
    And(0.34 <= flavanoids, 5.08 >= flavanoids),
    And(0.13 <= nonflavanoid_phenols,
        0.66 >= nonflavanoid_phenols),
    And(0.41 <= proanthocyanins, 3.58 >= proanthocyanins),
    And(1.28 <= color_intensity, 13 >= color_intensity),
    And(0.48 <= hue, 1.71 >= hue),
    And(1.27 <= od280/od315_of_diluted_wines,
        4 >= od280/od315_of_diluted_wines),
    And(278 <= proline, 1680 >= proline))


In [526]:
print(explainer.T_model)

And(And(Implies(And(proline < 770), o_0_0 == 0.140779421),
        Implies(And(proline >= 770, flavanoids < 2.28999996),
                o_0_0 == 0.0646421313),
        Implies(And(proline >= 770,
                    flavanoids >= 2.28999996),
                o_0_0 == -0.270458966)),
    And(Implies(And(proline < 770), o_1_0 == 0.134412304),
        Implies(And(proline >= 770, flavanoids < 2.17000008),
                o_1_0 == 0.0870177448),
        Implies(And(proline >= 770,
                    flavanoids >= 2.17000008,
                    magnesium < 126),
                o_1_0 == -0.236294851),
        Implies(And(proline >= 770,
                    flavanoids >= 2.17000008,
                    magnesium >= 126),
                o_1_0 == -0.0528627001)),
    And(Implies(And(proline < 770), o_2_0 == 0.129026279),
        Implies(And(proline >= 770, flavanoids < 2.17000008),
                o_2_0 == 0.0835888311),
        Implies(And(proline >= 770,
                    flavanoids >= 

In [527]:
xgbc.predict([X_test.values[0]], output_margin=True)

array([-1.9658402], dtype=float32)

In [528]:
-0.265196383 + -0.225890428 + -0.199127138 + y.mean()

np.float64(-0.021674623157303397)

In [529]:
print(explainer.label_proportions)

0.6685393258426966


In [530]:
print(explainer.D)

And(o_0_0 +
    o_1_0 +
    o_2_0 +
    o_3_0 +
    o_4_0 +
    o_5_0 +
    o_6_0 +
    o_7_0 +
    o_8_0 +
    o_9_0 +
    o_10_0 +
    o_11_0 +
    o_12_0 +
    o_13_0 +
    o_14_0 +
    o_15_0 +
    o_16_0 +
    o_17_0 +
    o_18_0 +
    o_19_0 +
    0.6685393258? <
    0)


In [531]:
print(explainer.I)

[alcohol == 13.63, malic_acid == 1.81, ash == 2.7, alcalinity_of_ash == 17.2, magnesium == 112, total_phenols == 2.85, flavanoids == 2.91, nonflavanoid_phenols == 0.3, proanthocyanins == 1.46, color_intensity == 7.3, hue == 1.28, od280/od315_of_diluted_wines == 2.88, proline == 1310]


In [532]:
if explainer.delta_expressions:
    print(explainer.delta_expressions)

[flavanoids >= 2.91 - delta, flavanoids <= 2.91 + delta, proline >= 1310 - delta, proline <= 1310 + delta, delta >= 0]
